In [ ]:
from data_loader import generate_data
from instruct_prompt_generator import load_ontologies, generate_instruct_prompt
import re
def pretty_print(message):
    print('\n\n'.join('\n'.join(line.strip() for line in re.findall(r'.{1,100}(?:\s+|$)', paragraph.strip('\n'))) for paragraph in re.split(r'\n\n+', message)))



train_path = "/content/drive/MyDrive/Prompt_Semantic/Data/karma-version/sources/"
models_path = "/content/drive/MyDrive/Prompt_Semantic/Data/models-triples/"
ontology_path = "/content/drive/MyDrive/Prompt_Semantic/Data/ontology/Ontologies.json"


VARIABLES = ["Table", "Ontology"]
variable_string = ""
for variable in VARIABLES:
    variable_string += "\n{$" + variable.upper() + "}"
print(variable_string)

train_data, test_data = generate_data(train_path, models_path,  6, 3)
ontology = load_ontologies(ontology_path)
variable_values = {}
variables = {'$EXAMPLES', '$ONTOLOGY', '$TABLE'}
for variable in variables:
    print("Enter value for variable:", variable)
    if variable == "$TABLE":
        variable_values[variable] = str(test_data[0]['table'])
    if variable == "$EXAMPLES":
        examples = ""
        for example_data in train_data[:3]:
          examples += f"<Table> {example_data['table']} </Table>\n"
          examples += "<SemanticGraph>"
          # examples += f"<Step1> {example_data['semantic_graph']['SetSemanticType']} </Step1>\n"
          # examples += f"<Step2> {example_data['semantic_graph']['SetInternalLink']} </Step2>\n"
          examples += f"{example_data['semantic_graph']} </SemanticGraph>\n"
          variable_values[variable] = examples
    if variable == "$ONTOLOGY":
        nodes = []
        properties = []
        potential_triples = []
        ontologies = ""
        for ontology_name, ontology_data in ontology.items():
            for node in ontology_data['Nodes']:
                nodes.append(node)
            for prop in ontology_data['Properties']:
                properties.append(prop)
            for triples in ontology_data['Potential triples']:
                potential_triples.append(triples.replace("<", "[").replace(">", "]"))
        ontologies += f"<Nodes> {nodes}</Nodes> \n <Properties> {properties}</Properties> \n <PotentialTriples> {potential_triples}</PotentialTriples>\n"
        variable_values[variable] = ontologies


In [ ]:
with open("PromptTemp.txt", "r") as f:
    extracted_prompt_template = f.read()

prompt_with_variables = extracted_prompt_template
for variable in variable_values:
    prompt_with_variables = prompt_with_variables.replace("{" + variable + "}", variable_values[variable])

promtp = open("prompt.txt", "w")
promtp.write(prompt_with_variables)
promtp.close()



model = ""
message = model.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                "role": "user",
                "content":  prompt_with_variables
            },
        ],
        stream=False,
    ).choices[0].message.content

print("Claude's output on your prompt:\n\n")
pretty_print(message)